In [1]:
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

%run metrics.ipynb

In [13]:
def NaiveTime(data):
    """
    Naive time algorithm, predicts the time untill the next event, 
    based on the average time between two events
    param: data
    return: print of metrics
    """
    train_Naive(data)
    y_test, y_pred = feature_selection_time(data)
    time_metrics(y_test, y_pred, "Naive Time")


def train_Naive(data):
    """
    Using a vectorization techniques to calculate and predict time
    """
    data["next"] = (data['event time:timestamp'] - data['event time:timestamp'].shift()
                    ).fillna(pd.Timedelta(seconds=0)).reset_index(drop=True)
    data["naive_time"] = data["next"].agg("mean")
    data["naive_time"] = data["naive_time"].dt.seconds
    data["delta_time"] = (data["next_time"] - data["event time:timestamp"])
    data["delta_time"] = data["delta_time"].dt.seconds


def feature_selection_time(data):
    """
    Selects and creates the features to calculate metrics
    """
    x = data.dropna(subset=["delta_time"])
    y = x[["delta_time", "naive_time"]]
    y.delta_time = y.delta_time.astype(int)

    return y.delta_time, y.naive_time

    

In [10]:
def NaiveEvent(data):
    """
    Naive event algorithm that predicts the next event based on the previous
    param: Data
    return: print of metrics
    """
    simplePred(data)
    y_test, y_pred = pred_encoder(data)
    event_metrics(y_test, y_pred, "Naive Event")
    
    

def simplePred(data, cEvnt="event concept:name", cNw="next_event", cPrd="naive_next_event"):
    """
    Loops through the list of events and adds them to the df
    """
    lst = data[cEvnt].unique().tolist()
    i = 0
    while i < len(lst):
        data.loc[data[cEvnt] == lst[i], cPrd] = data[data[cEvnt] == lst[i]][cNw].mode()[0]
        i += 1

def pred_encoder(data):
    """
    Encodes the data so that it can be used for prediction calculations
    """
    event_encoder = LabelEncoder()
    labels = data["next_event_log"].unique()
    event_encoder.fit(labels)
    y_test = event_encoder.transform(data[["next_event_log"]])
    y_pred = event_encoder.transform(data[["naive_next_event"]])    
    return y_test, y_pred
    